In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import seaborn as sns
from tabulate import tabulate
from typing import Optional
from functions_dependency_test import find_dependent_chi, p_of_chi_squared, heatmap_chi
from Data_transformations import feature_cleaning, values_cleaning, missing_values_removing
plt.style.use("seaborn")

In [40]:
df = pd.read_csv('Murder_Data.zip', index_col=0, compression='zip')
df.insert(1, 'County', df['CNTYFIPS'].map(lambda x : str(x).split(',')[0]), True)
df.drop("CNTYFIPS",axis = 1, inplace= True) # redundant, onlyy keeping the county part
df.drop("StateName",axis = 1, inplace= True)  # redundant, contains state
df.drop("Ori",axis = 1, inplace= True)  # redundant, contains state + county + agentcy nb
df.insert(3, 'Area',df['MSA'].map(lambda x :  str(x).split(',')[0] ) ,True)
df.drop('MSA',axis = 1,inplace=True) # rendundant only the state, keeping the metropolitant statistical area
# split File Date in Day-Month-Year columns
nan_indexes = df["FileDate"].isna()
df['FileDate'].fillna(0,inplace = True)
df['FileDate'] = df['FileDate'].astype(int)
df['FileDate'] = df['FileDate'].astype(str)
df.insert(1, 'File Year', df['FileDate'].map(lambda x : x[len(x)-2:len(x)]), True)
df.insert(1, 'File Day', df['FileDate'].map(lambda x : x[len(x)-4:len(x)-2]), True)
df.insert(1, 'File Month', df['FileDate'].map(lambda x : x[0:len(x)-4]), True)
df.loc[nan_indexes,"File Year"] = np.NaN
df.loc[nan_indexes,"File Day"] = np.NaN
df.loc[nan_indexes,"File Month"] = np.NaN
df.drop("FileDate",axis = 1,inplace = True)
df.drop("Incident",axis = 1,inplace = True) # incient number of the month, for a specific county useless for our application
df.drop("Situation",axis = 1,inplace = True) # redundant, info already contained in OffCount and VicCount

# data_pp = values_cleaning(feature_cleaning(data_original))
# data_pp.head()

# drop ID column, as it has too many values -> computation of Chi-squared takes too long
data = pd.read_csv('non_empty_sample.csv', index_col=0)
data_mod = data.drop(['ID'], axis = 1)

/Users/thomas/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [33]:
df_heat = heatmap_chi(data_mod)

,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,StateName,Month,...,OffSex,OffRace,Weapon,Relationship,Circumstance,VicCount,OffCount,FileDate,fstate,MSA
CNTYFIPS,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.925932e-180,...,2.071277e-263,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0
Ori,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.522710e-320,...,1.325600e-186,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0
State,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.703581e-95,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0
Agency,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.983381e-248,...,2.084418e-180,0.0,0.0,0.0,0.0,0.0,2.859316e-139,0.0,0.0,0.0
Agentype,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.707776e-08,...,6.249890e-18,0.0,0.0,0.0,0.0,0.0,2.107465e-104,0.0,0.0,0.0


In [41]:
df_test = heatmap_chi(df)

,County,File Month,File Day,File Year,State,Agency,Area,Agentype,Source,Solved,...,OffAge,OffSex,OffRace,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount
County,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
File Month,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1.559583e-72,0.000000e+00,0.000000e+00
File Day,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1.189626e-93,0.000000e+00,0.000000e+00
File Year,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
State,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
Agency,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,7.435482e-08
Area,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
Agentype,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,4.229583e-87,0.000000e+00,6.611757e-27
Source,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.033900e-02,...,0.000000e+00,2.342146e-27,9.029114e-66,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1.496100e-73,5.548746e-74,4.867762e-130
Solved,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,1.033900e-02,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,2.479182e-203,5.232424e-159,0.000000e+00
